In [1]:
import os
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from pyimzml.ImzMLParser import ImzMLParser, _bisect_spectrum
from typing import List, Callable

%matplotlib inline

In [2]:
# set path containing the DHG dataset
data_p = "/sise/assafzar-group/assafzar/Leor/DHG/Cropped/TIC-normalized/"

In [3]:
# get all mass spec images names
img_names = np.asarray([f for f in os.listdir(data_p) if f.endswith('.imzML')])

In [4]:
def get_ion_image(p: ImzMLParser,
                  mz_values: List[float],
                  tol: float = 0.1,
                  z: int = 1,
                  reduce_func: Callable = sum) -> np.ndarray:
    """
    Get an image representation of the intensity distribution
    of the ion with specified m/z values.

    By default, the intensity values within the tolerance region are summed.

    Args:
        p (ImzMLParser): the ImzMLParser (or anything else with similar
        attributes) for the desired dataset.
        mz_values (List[float]): m/z values for which the ion image
        shall be returned
        tol (float, optional): Absolute tolerance for the m/z value,
        such that all ions with values mz_value-|tol| <= x <= mz_value+|tol|
        are included. Defaults to 0.1
        z (int, optional): z Value if spectrogram is 3-dimensional.
        Defaults to 1.
        reduce_func (Callable, optional): the bahaviour for reducing the
        intensities between mz_value-|tol| and mz_value+|tol| to a single
        value. Must be a function that takes a sequence as input and outputs
        a number. By default, the values are summed. Defaults to sum.

    Returns:
        [np.ndarray]: numpy matrix with each element representing the
        ion intensity in this pixel. Can be easily plotted with matplotlib

    """
    tol = abs(tol)
    max_y = p.imzmldict["max count of pixels y"]
    max_x = p.imzmldict["max count of pixels x"]
    ims = [np.zeros((max_y, max_x)) for _ in range(0, len(mz_values))]
    for i, (x, y, z_) in enumerate(p.coordinates):
      if z_ == 0:
        UserWarning(("z coordinate = 0 present, if you're getting blank "
                     "images set getionimage(.., .., z=0)"))
      if z_ == z:
        mzs, ints = map(np.asarray, p.getspectrum(i))
        for j, mz_value in enumerate(mz_values):
          im = ims[j]
          min_i, max_i = _bisect_spectrum(mzs, mz_value, tol)
          im[y - 1, x - 1] = reduce_func(ints[min_i:max_i + 1])
    return ims

In [5]:
# define the m/z values we wish to visualize
plot_mz = [749.5, 834.5, 888.6]
# define the tolerance for retrieving an m/z value
plot_tol = 0.025
# define plot full brithness percenbtage treshould
plot_tresh = 95

In [6]:
# delte and create output folder
!rm -rf Output
!mkdir Output

In [7]:
# loop over each mass spec image name and plot mz ion image
for img_name in img_names:
  # Parse the mass spec image imzML file
  with ImzMLParser(os.path.join(data_p, img_name)) as p_l:
    # get ion image representation for each m/z value
    ims_l = get_ion_image(p_l, plot_mz, tol=plot_tol)
    # get mass spec image imzML file name without file suffix
    img_name_l = os.path.splitext(img_name)[0]
    
    # loop over each ion image + sum/average and plot
    for mz, im_l in zip(plot_mz + ["MZ_Sum"], ims_l + [sum(ims_l)]):  
      # plot mz ion image
      plt.figure(frameon=False)
      plt.imshow(im_l, vmin=im_l.min(), vmax=np.percentile(im_l, plot_tresh),
                 cmap='viridis')
      plt.axis("off")

      # save plot
      plt.savefig(f"Output/{img_name_l}-{mz}.png", transparent=True,
                  bbox_inches="tight", pad_inches=0)
      
      # clear plot
      plt.close("all")